In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install sklearn
!pip install seaborn

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_set = pd.read_csv("train.csv")
test_set = pd.read_csv("test.csv")
pokemons = pd.read_csv("pokemon.csv")
train_set["train"] = 1
test_set["train"] = 0
all_data = pd.concat([train_set, test_set])

In [ ]:
all_data.info()

In [ ]:
pokemons.describe(include="object")

Droppo le colonne dei nomi, in quanto valori univoci non utili

In [5]:
pokemons.drop(labels=["Name"], axis=1, inplace=True)

# Facciamo il join delle due tabelle di dati, in quanto le caratteristiche dei pokemon serviranno per allenare il modello

In [6]:
joined = all_data.join(pokemons.set_index("#"), on="First_pokemon", rsuffix="_first")
joined = joined.join(pokemons.set_index("#"), on = "Second_pokemon", rsuffix="_second")


In [ ]:
joined.head()

In [ ]:
joined.info()

# Per utilizzare un classificatore binario, reinterpreto la colonna Winner in questo moodo:
## 1 -> Ha vinto il primo pokemon
## 0 -> Ha vinto il secondo pokemon

In [9]:
def binary_winner(winner, first_pokemon, second_pokemon):
  return 1 if (first_pokemon == winner) else 0

In [ ]:
joined["Winner"] = joined.apply(lambda x: binary_winner(x['Winner'], x['First_pokemon'], x['Second_pokemon']), axis=1)
joined.head()

In [11]:
joined.drop(labels=["First_pokemon", "Second_pokemon"], axis=1, inplace=True)

# ANALISI DEI VALORI NULLI

In [ ]:
joined.info()

In [ ]:
pokemons.info()

Poichè contiene molti valori nulli, decido di droppare la colonna con il Second Type

In [14]:
joined.drop(labels=["Type 2", "Type 2_second"], axis=1, inplace=True)


# Encoding

In [15]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
joined["Legendary"] = encoder.fit_transform(joined["Legendary"])
joined["Legendary_second"] = encoder.fit_transform(joined["Legendary_second"])
joined["Type 1"] = encoder.fit_transform(joined["Type 1"])
joined["Type 1_second"] = encoder.fit_transform(joined["Type 1_second"])

In [ ]:
joined.head()

# Counting

In [ ]:
sns.countplot(x="Legendary", hue="Winner", data=joined)
plt.show()
sns.countplot(x="Legendary_second", hue="Winner", data=joined)
plt.show()
sns.countplot(x="Generation", hue="Winner", data=joined)
plt.show()
sns.countplot(x="Generation_second", hue="Winner", data=joined)
plt.show()

Dai grafici riportati notiamo che la distribuzione dell'attributo di Genertion è abbastanza uniforme, perciò avrà probabilmente senso droppare questa feature. La conferma la avremo dalla heatmap.

In [ ]:
joined["Legendary_diff"] = joined.Legendary - joined.Legendary_second
sns.countplot(x="Legendary_diff", hue="Winner", data=joined)
plt.show()

Per i valori di Legendary invece possiamo notare come quando i due pokemon appartengono alla stessa classe (Legendary_diff = 0) allora non c'è troppa differenza tra la percentuale di vittorie o sconfitte. Mentre quando uno dei due è leggendario e l'altro no, la battaglia volge a favore del leggendario

# Analisi delle correlazioni

Consideriamo anche la correlazione tra il valore di output e le differenze dei valori delle statistiche

In [19]:
joined["Atk_diff"] = joined.Attack - joined.Attack_second
joined["Def_diff"] = joined.Defense - joined.Defense_second
joined["Spd_diff"] = joined.Speed - joined.Speed_second
joined["Atk_sp_diff"] = joined["Sp. Atk"] - joined["Sp. Atk_second"]
joined["Def_sp_diff"] = joined["Sp. Def"] - joined["Sp. Def_second"]
joined["Hp_diff"] = joined.HP - joined.HP_second

In [ ]:
plt.figure(figsize=(25,10))
sns.heatmap(joined.corr(), annot=True, linewidths=2)

Data la bassa correlazione con "Type 1", "Generation", "Generation_second" decido di dropapre queste tre colonne
Le correlazioni qui espresse sono coerenti con quanto già si poteva notare dai grafici countplot: la statistica di Generation è poco correlata, mentre Legendary ha una leggera correlazione in più e può concorrere alla classificazione.

In [21]:
joined.drop(labels=["Type 1", "Generation", "Generation_second"], axis=1, inplace=True)

Dato che la Legendary_diff ha una correlazione simile a quella di Legendary e Legendary_second elimino anche queste due feature condensando l'informazione solo sul fatto che la classe sia diversa o no.

In [22]:
joined.drop(labels=["Legendary", "Legendary_second"], axis=1, inplace=True)

# Training

In [ ]:
joined.head()

In [24]:
train_set = joined[joined.train == 1].drop("train", axis=1)
test_set = joined[joined.train == 0].drop("train", axis=1)

y_train = train_set.Winner
x_train = train_set.drop("Winner", axis=1)
y_test = test_set.Winner
x_test = test_set.drop("Winner", axis=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix, accuracy_score

model = RandomForestClassifier()
model = model.fit(x_train, y_train)
    
y_pred = model.predict(x_test)
accuracy = round(accuracy_score(y_pred, y_test) * 100, 2)
print("Accuracy: ", accuracy)
  
plot_confusion_matrix(model, x_test, y_test)
plt.show()